# Resource Validation

Created for air-gapped clouds, this notebook validates whether the VBD resources in this solution are available and configured in the current subscription.  

### Python Prerequisite Libraries for Validation

The following libraries may need to be installed in your kernel to validate the Azure session.  If you do NOT have a connection to the general web, you will need to find a secure means to load these packages on your own Python kernel (using Terminal for your compute instance is the recommended way).

In [ ]:
# verification as to whether packages above are installed or not
import subprocess as sp
import json

from pathlib import Path

verifyPkgs = []
installedPkgs = []
splitChars = [ "==", ">=", "<=", ">", "<" ]

# check for important and necessary packages/modules
filePath = "./common/requirements.txt"
reqFilePath = Path( filePath )

if reqFilePath.exists() :

    with open( filePath, "r" ) as f :

        requirements = [ line.strip() for line in f if line.strip() ]

        for requirement in requirements :

            verifyPkgs.append( requirement )

# load currently installed packages
pkgs = sp.check_output( [ "pip", "list", "--format", "json" ] )
pkgsJson = json.loads( pkgs )

# print( pkgsJson )

for pkg in pkgsJson :

    installedPkgs.append( pkg[ "name" ] )

# load package list to verify installation
for checkPkg in verifyPkgs :

    pName = checkPkg
    pVer = ""
    pChar = ""

    for sc in splitChars :

        if sc in checkPkg :

            pkgSplit = checkPkg.split( sc )

            if len( pkgSplit ) > 1 :

                pName = pkgSplit[ 0 ]
                pVer = pkgSplit[ 1 ]
                pChar = sc

            break

    if pName not in installedPkgs :

        # cmdList = [ "pip", "install", "--upgrade", pName + sc + pVer ]

        print( "*** Attempting to install missing package " + checkPkg + " ***" )

        if pVer != "" :

            cmdList = [ "pip", "install", "--upgrade", pName + pChar + pVer ]

        else :

            cmdList = [ "pip", "install", checkPkg ]

        print( cmdList )
        spOutput = sp.run( cmdList, capture_output = True )

        print( spOutput )

    else :

        print( checkPkg + " already installed." )

### Python Package Load

These packages will be used to validate the resources in this Azure subscription.

In [ ]:
from azureml.core import Workspace
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
import pkg_resources

from azure.mgmt.search import SearchManagementClient
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

import requests
import os
import json
from dotenv import load_dotenv

try:
    load_dotenv(dotenv_path="credentials.env")
    print("Successfully loaded environment variables from credentials.env")
    
except Exception as e:
    print(f"An error occurred while loading environment variables from credentials.env: {e}")

In [ ]:
# Uncomment this line if you want to check the packages currently installed
# !pip freeze

### Initialize Resource Management Client

Accessing the Azure Resource Management Client will be needed in further validation steps.

See documentation at https://learn.microsoft.com/en-us/python/api/overview/azure/identity-readme?view=azure-python

In [ ]:
# Initialize Azure credentials
from azure.identity import AzureCliCredential
credential = AzureCliCredential()

# Initialize a Workspace object from the existing workspace you have
ws = Workspace.from_config()

# Retrieve and print the subscription ID
subscription_id = ws.subscription_id
print("Subscription ID:", subscription_id)

# Initialize the Resource Management client
resource_client = ResourceManagementClient(credential, subscription_id)

### Validation:  Azure Cognitive Search

Check whether the credentials.env settings for Azure Cognitive Search lead to an active session.

In [ ]:
def check_azure_search_service(api_version, endpoint, search_key):
    headers = {
        'api-key': search_key,
        'Content-Type': 'application/json'
    }
    params = {'api-version': api_version}

    # Construct the URL for the request
    url = f"{endpoint}/servicestats"

    try:
        # Make the API request
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            print("Azure Cognitive Search service is active.  The following output shows the service statistics.")
        else:
            print(f"Received status code {response.status_code}. Service may not be active or the request was invalid.")

        # Parse the JSON content
        json_data = json.loads(response.content)

        # Pretty-print the JSON data
        pretty_str = json.dumps(json_data, indent=4)
        print(pretty_str)

    except Exception as e:
        print(f"An error occurred: {e}")

# Set your own API version, endpoint, and search key
api_version = os.environ['AZURE_SEARCH_API_VERSION']
endpoint = os.environ['AZURE_SEARCH_ENDPOINT']
search_key = os.environ['AZURE_SEARCH_KEY']

check_azure_search_service(api_version, endpoint, search_key)

#### Semantic Search

Semantic search for Azure Cognitive Search may not be available for your region, and may not be enabled for your search service.
The best way to check is to use the Azure Portal using instructions from https://learn.microsoft.com/en-us/azure/search/semantic-how-to-enable-disable?tabs=enable-portal#enable-semantic-search

The cell below will validate whether or not the Search service being used had Semantic available or not, whether by simply not being enabled or not being available.

In [ ]:
# Ignore temporarily
'''
import common.utils as cu

searchService = os.environ[ 'AZURE_SEARCH_ENDPOINT' ]
resourceGroup = os.environ[ 'AZURE_SEARCH_RG' ]
subscriptionId = os.environ[ "AZURE_SEARCH_SUB_ID" ]

if cu.semanticEnabled( searchService, subscription_id, resourceGroup ) :

    print( "Semantic Search is enabled." )

else :

    print( "Semantic Search is either disabled or not available for the Search Service instance provided." )
'''

#### 